# Capstone: Epidemic propagation

## Objectives
Can you run a simple spatial-temporal clustering of recent historic epidemics, and can this give you the next epidemics?

Can you pull in some main socio-economic features of the centroids of these clusters and run PCA to see the commonalities across these. What do we predict through this?

Can you include propagation of epidemics and see how they spread?

Therefore... 
Is the Coronavirus predictable?
Where's the next one, and how will that spread?
    
## Data required:
1. Recent epidemics to consider: SARS, Coronavirus, MERS, Ebola, Zika, Bird Flu
2. Socio-economic data for each virus: Long / Lat, Population, Development index, cleanliness etc.?
3. Epidemic propagation: airport traffic at each infected city, volume of travel between cities affected

In [3]:
import pandas as pd